# Detección de características

Dado que las esquinas son características interesantes de una imagen. Los algoritmos de detección de características comenzaron con la detección de esquinas. Hay varias técnicas en OpenCV para detectar las características.
<br>
Detección de esquinas Haris
<br>
Detección de esquinas Shi-Tomasi
<br>
SIFT (Transformación de características de escala invariable)
<br>
SURF (Funciones robustas aceleradas)
<br>
Algoritmo FAST para la detección de esquinas
<br>
ORB (Breve orientado FAST y girado)
<br><br>
SIFT, SURF están patentados y no están disponibles gratuitamente para uso comercial. Requiere opencv-contrib instalado para poder usarlos

## Detección de esquinas Haris

In [9]:
import cv2
import numpy as np

img = cv2.imread('images/scene.jpg')

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_img = np.float32(gray_img)

dst = cv2.cornerHarris(gray_img, blockSize=2, ksize=3, k=0.04)

# dilate to mark the corners
dst = cv2.dilate(dst, None)
img[dst > 0.01 * dst.max()] = [0, 255, 0]

cv2.imshow('haris_corner', img)
cv2.waitKey()

-1

## Detección de esquinas Shi-Tomasi
Shi y Tomasi idearon una función de puntuación diferente a la utilizada en el detector de esquinas de Haris para encontrar N esquinas más acentuadas a partir de una imagen.

In [10]:
import cv2
import numpy as np

img = cv2.imread('images/scene.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

corners = cv2.goodFeaturesToTrack(gray_img, maxCorners=50,      qualityLevel=0.02, minDistance=20)
corners = np.float32(corners)

for item in corners:
    x, y = item[0]
    cv2.circle(img, (x, y), 6, (0, 255, 0), -1)

cv2.imshow('good_features', img)
cv2.waitKey()

<ipython-input-10-414cb99b1115>:12: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.circle(img, (x, y), 6, (0, 255, 0), -1)


-1

Las dos técnicas anteriores Haris Corner and Shi-Tomasi son invariantes en la rotación, lo que significa que incluso si se giran las esquinas, seremos capaces de detectarlas con exactitud. Sin embargo, son variantes a escala; si se amplían las esquinas perderemos la forma en la región seleccionada y los detectores no podrán identificarlas.

## SIFT
SIFT es invariante tanto en rotación como en escala. SIFT proporciona puntos clave y descriptores de puntos clave donde el descriptor de puntos clave describe el punto clave en una escala seleccionada y rotación con gradientes de imagen.
<br>
<br>
La imagen resultante tiene círculos que representan los puntos / características clave, donde el tamaño del círculo representa la fuerza del punto clave y la línea dentro del círculo denota la orientación del punto clave.

In [12]:
import cv2

img = cv2.imread('images/scene.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.xfeatures2d.SIFT_create()
kp, des = sift.detectAndCompute(gray_img, None)

kp_img = cv2.drawKeypoints(img, kp, None, color=(0, 255, 0),                    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('SIFT', kp_img)
cv2.waitKey()

-1

## SURF (Funciones robustas aceleradas)
Aunque SIFT funciona bien, realiza operaciones intensivas que requieren mucho tiempo. SURF se introdujo para tener todas las ventajas de SIFT con un tiempo de procesamiento reducido.

In [13]:
import cv2

img = cv2.imread('images/scene.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

surf = cv2.xfeatures2d.SURF_create()
kp, des = surf.detectAndCompute(gray_img, None)

kp_img = cv2.drawKeypoints(img, kp, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('SURF', kp_img)
cv2.waitKey()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wwma2wne\opencv_contrib\modules\xfeatures2d\src\surf.cpp:1029: error: (-213:The function/feature is not implemented) This algorithm is patented and is excluded in this configuration; Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in function 'cv::xfeatures2d::SURF::create'


## Algoritmo FAST para la detección de esquinas
SURF es rápido en comparación con SIFT, pero no tan rápido para usarlo con dispositivos en tiempo real como teléfonos móviles. Así que se introdujo el algoritmo FAST con un tiempo de procesamiento reducido. Sin embargo, FAST nos da solo los puntos clave y es posible que necesitemos calcular descriptores con otros algoritmos como SIFT y SURF.

In [14]:
import cv2

img = cv2.imread('images/scene.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

fast = cv2.FastFeatureDetector_create()
fast.setNonmaxSuppression(False)

kp = fast.detect(gray_img, None)
kp_img = cv2.drawKeypoints(img, kp, None, color=(0, 255, 0))

cv2.imshow('FAST', kp_img)
cv2.waitKey()

-1

## ORBE
ORB es una alternativa eficiente de código abierto a SIFT y SURF. A pesar de que calcula menos puntos clave en comparación con SIFT y SURF, son efectivos. Utiliza técnicas FAST y BRIEF para detectar los puntos clave y calcular los descriptores de imagen respectivamente.

In [15]:
import cv2

img = cv2.imread('images/scene.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

orb = cv2.ORB_create(nfeatures=2000)
kp, des = orb.detectAndCompute(gray_img, None)

kp_img = cv2.drawKeypoints(img, kp, None, color=(0, 255, 0), flags=0)

cv2.imshow('ORB', kp_img)
cv2.waitKey()

-1

## Apareamiento de características
La coincidencia de características entre imágenes en OpenCV se puede realizar con un comparador de fuerza bruta o un comparador basado en FLANN.
### Comparador de fuerza bruta (BF)
BF Matcher compara el descriptor de una característica de una imagen con todas las demás características de otra imagen y devuelve la coincidencia en función de la distancia. Es lento ya que verifica que coincida contra todas las caracterìsticas.

In [19]:
import cv2

img1 = cv2.imread('images/book_cover.jpg', 0)
img2 = cv2.imread('images/book_cover_rotated.jpg', 0)

orb = cv2.ORB_create(nfeatures=500)
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

# matcher takes normType, which is set to cv2.NORM_L2 for SIFT and SURF, cv2.NORM_HAMMING for ORB, FAST and BRIEF
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)
# draw first 50 matches
match_img = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None)
cv2.imshow('Matches', match_img)
cv2.waitKey()

-1

### FLANN based matcher
La biblioteca rápida para vecinos más cercanos aproximados (FLANN) está optimizada para encontrar las coincidencias con la búsqueda incluso con grandes conjuntos de datos, por lo que es rápida en comparación con el comparador de fuerza bruta.
<br>
Con ORB y FLANN matcher extraemos la portada del libro tesla de la segunda imagen y corrijamos la rotación con respecto a la primera imagen

In [22]:
import argparse

import cv2
import numpy as np

im1 = cv2.imread('images/book_cover.jpg', 0)
im2 = cv2.imread('images/book_cover_rotated.jpg', 0)

img = get_corrected_img(im2, im1)
cv2.imshow('Corrected image', img)
cv2.waitKey()

def get_corrected_img(img1, img2):
    MIN_MATCHES = 50

    orb = cv2.ORB_create(nfeatures=500)
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    index_params = dict(algorithm=6,
                        table_number=6,
                        key_size=12,
                        multi_probe_level=2)
    search_params = {}
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # As per Lowe's ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) > MIN_MATCHES:
        src_points = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_points = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        m, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
        corrected_img = cv2.warpPerspective(img1, m, (img2.shape[1], img2.shape[0]))

        return corrected_img
    return img2